---
title: "Visualization (Exploring Co-variation)"
author: "Peter Ganong and Maggie Shi"
date: today
date-format: long
format: 
    html:
        echo: true
        code: true
        toc: true
        output: true
engine: jupyter
---

In [ ]:
#| echo: false
#| warning: false
import altair as alt
from vega_datasets import data
from palmerpenguins import load_penguins
penguins = load_penguins()
from plotnine import *
from plotnine.data import diamonds, mpg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
alt.data_transformers.disable_max_rows() # Needed because len(df) > 5000

## Table of contents
1. Categorical variable and a continuous variable
2. Two categorical variables
3. Two continuous variables
4. Graphics for production



# Categorical variable and continuous variable
<!-- source material (lecture 1, slides 33-36) -->

<!-- source: Lecture 5, slides 45-65)-->


In [ ]:
from palmerpenguins import load_penguins
penguins = load_penguins()
display(penguins)

## numeric & categorical: box plot

![](pictures/EDA-boxplot.png)

## numeric & categorical: `mark_boxplot()` 

In [ ]:
alt.Chart(penguins).mark_boxplot().encode(
    x=alt.X('species:N', title="Species"), 
    y=alt.Y('body_mass_g:Q', title="Body Mass (g)"),
).properties(
    width=400,
    height=300
)

Discussion question: what do you notice from this graph? 


## numeric & categorical: `transform_density()` 

In [ ]:
alt.Chart(penguins).transform_density(
    'body_mass_g',
        groupby=['species'], 
        as_=['body_mass_g', 'density']
    ).mark_line().encode(
        alt.X('body_mass_g:Q'),
        alt.Y('density:Q', stack=None), 
        alt.Color('species:N')
    ).properties(width=400,height=300)

## numeric & categorical: `transform_density()` 

Discussion q -- What if we required the x-axis range to include zero? Would that improve or reduce clarity? How come? 

In [ ]:
alt.Chart(penguins).transform_density(
    'body_mass_g',
        groupby=['species'],  
        as_=['body_mass_g', 'density']
    ).mark_line().encode(
        alt.X('body_mass_g:Q', scale=alt.Scale(zero=True)),
        alt.Y('density:Q', stack=None), 
        alt.Color('species:N')
    ).properties(width=400,height=300)

## numeric & categorical: `transform_density()` filled in

`opacity=0.3` makes no difference in content; maybe a bit more elegant

In [ ]:
alt.Chart(penguins).transform_density(
    'body_mass_g',
        groupby=['species'],  # Group by species for different density curves
        as_=['body_mass_g', 'density']
    ).mark_area(opacity=0.3).encode(
        alt.X('body_mass_g:Q'),
        alt.Y('density:Q', stack=None), 
        alt.Color('species:N')
    ).properties(width=400,height=300)

# Two categorical variables

## Question: How is cut related to color? 2 categorical vars

In [ ]:
#| warning: false
diamonds_grouped = diamonds.groupby(['color','cut']).size().reset_index().rename(columns={0:'N'})
diamonds_grouped

## Question: How is cut related to color? 2 categorical vars

In [ ]:
alt.Chart(diamonds_grouped).mark_circle().encode(
    x = 'color:N',
    y = 'cut:N',
    size='N:Q')

## Question: How is cut related to color? 2 categorical vars

In [ ]:
alt.Chart(diamonds_grouped).mark_rect().encode(
    x = 'color:N',
    y = 'cut:N',
    color='N:Q')

Discussion question: what diamond types are most common?



# Two continuous variables

## Two continuous variables: roadmap

* `movies` ratings from Rotten Tomatoes and IMDB 
* `diamonds`: `carat` vs `price`

## movies dataset

In [ ]:
movies_url = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'

In [ ]:
#| eval: false
movies = pd.read_json(movies_url)

In [ ]:
#| echo: false
movies = pd.read_json('data/movies.json')

## Covariation: a first binned scatter plot

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('IMDB_Rating:Q', bin=alt.BinParams(maxbins=20)),
)

Suffers from overplotting!


## use `alt.Size('count()')` to address overplotting

In [ ]:
xy_size = alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('IMDB_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Size('count()')
)
xy_size

## use `alt.Color('count()')` to address overplotting

In [ ]:
xy_color = alt.Chart(movies_url).mark_bar().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('IMDB_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Color('count()')
) 
xy_color

## Discussion question

In [ ]:
xy_size | xy_color

Compare the *size* and *color*-based 2D histograms above. Which encoding do you think should be preferred? Why?



## Question: How is carat related to price? 2 continuous vars

In [ ]:
alt.Chart(diamonds).mark_point().encode(
    x = 'carat:Q',
    y = 'price:Q'
)

## Question: How is carat related to price? 2 continuous vars

In [ ]:
alt.Chart(diamonds).mark_rect().encode(
    alt.X('carat:Q', bin=alt.Bin(maxbins=70)), 
    alt.Y('price:Q', bin=alt.Bin(maxbins=70)),
    alt.Color('count()', scale=alt.Scale(scheme='blues')))

## Question: How is carat related to price? 2 continuous vars

In [ ]:
alt.Chart(diamonds).mark_boxplot().encode(
    alt.X('carat:Q', bin=alt.Bin(maxbins=10)), 
    alt.Y('price:Q'))

## Question: How is carat related to price? 2 continuous vars


In [ ]:
df = diamonds
df['carat_bin'] = pd.qcut(df['carat'], q=20, labels=(np.arange(1, 21, 1)))

df = df.groupby('carat_bin').agg(
    carat = ('carat', 'mean'),
    price = ('price', 'mean')).reset_index()

alt.Chart(df).mark_point().encode(
   x = 'carat:Q',
   y = 'price:Q'
)

* What it does:
    1. Computes bins using quantiles of x
    1. Computes means of y within each bin
    
* Called `binscatter` in stata and `binsreg` in R. Doesn't exist yet for Altair, but easy to code up yourself

## Discussion question -- "How is carat related to price?" 

Review the `mark_rect()`, `mark_boxplot()`, and `binscatter` plots

* headline? (aka the main message)
* sub-messages? (other information one can learn beyond the main message)


## Summary: Exploring covariation



| Scenario | Functions |
| ------- | --------- |
| Categorical and continuous variable | `mark_boxplot()` |
| | `transform_density()` |
| |     |
| Two categorical variables | `size` |
| | `color` |
| |       | 
| Two continuous variables | `alt.Size('count()')` |
| |  `alt.Color('count()')` |
| | `mark_boxplot()` |
| | binscatter |


<!--(lecture 1 slide 47 + lecture 6, slide 26-28 w/solution)-->

## Do-pair-share

We are now going to transition from making plots to teach **ourselves** to making plots for an audience.

Are penguins getting heavier (`body_mass_g`) over time? 

Bonus: what is the headline of your plot and what are the sub-messages?

<!-- ZZZ delete before posting -->
## Do-pair-share solution I


In [ ]:
alt.Chart(penguins).mark_bar().encode(
  alt.Y('average(body_mass_g):Q',  scale=alt.Scale(zero=False)),
  alt.X('year:N'),
  alt.Color('year:N')
)

This does answers the question, albeit in the most simple/boring way possible.

## Do-pair-share solution II

In [ ]:
alt.Chart(penguins).transform_density(
   'body_mass_g',
    groupby=['year'],
    as_= ['body_mass_g', 'density']
).mark_line().encode(
    x = 'body_mass_g:Q',
    y = 'density:Q',
    color='year:N'
)

* Headline: 2007 is lightest, 2008 is heaviest

* Sub-messages

    1. Similar shares of penguins above 5,000 grams in 2008 and 2009
    1. Average weight is higher in 2008 because 2009 has more lightweight penguins

<!-- ZZZ end deletion before posting -->

## Meta comment: iterating on plot design

"Make dozens of plots" -- Quoctrung Bui, former 30535 guest lecturer and former Harris data viz instructor

What does he mean?

* The first plot you make will never be the one you should show
* As a rule of thumb, you should try out at least three different plotting concepts (`mark`s)
* Within each concept, you will need to try out several different encodings
